# Problem Set, Part 2

This is the final segment of your problem set.

*** ENSURE YOU USE THE PROPER DATASET ***
To cut down on load time, drive usage, etc., I have provided you with a single year of Iowa liquor sales data (2019). The file is approx 140MB compressed and 625MB uncompressed. Note that Pandas is able to read compressed files. Here is the code for you to directly read a compressed file to a dataframe.
```Python
df = pd.read_csv('ILS_2019_on.csv.gz', compression = 'gzip')
```

I have added content to both the Pandas and Numpy notebooks to help you complete this set.

Here is the scenario. You have been tasked to assist with market research for Iowa liquor sales. Specifically, you have been asked to use the available data to answer the following three questions using the 2019 dataset referenced in the code above.

1. In terms of dollar amount sold, what are the top 5 liquor categories.

2. Based on bottles sold (monthly), display vodka and whiskey categories per quarter for 2019. You may assume that the category name include 'vodka' for all vodka products and the same for whiskey.

3. Again, based on bottles sold (monthly), which products were trending upward or downward (or flat)?

You will turn in this assignment as a notebook. The notebook should provide data to answer each question and be designed for stakeholders to consumer. In other words, your audience is not likely to be familiar with Python or programming in general. 

For up to 10% extra credit, provide relevant visualizations. Two useful libraries are `matplotlib` and `seaborn`. A `matplotlib` tutorial notebook has been provided in the course GitHub folder for you.

In [10]:
#imports
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
#file error handling
try:
    df = pd.read_csv('ILS_2019_on.csv.gz', compression = 'gzip')
except FileNotFoundError:
    print("Cannot find the file!")
#sorting the entire dataframe by date
df.sort_values(by = "Date", inplace = True)
#filtering out the 2020 data
df_date_filter = df[df["Date"].str.contains("2020")].index
df.drop(df_date_filter, inplace = True)
#converting date to datetime for later indexing
df['Date'] = pd.to_datetime(df['Date'])
#fixing spelling errors
df["Category Name"].replace({"American Cordials & Liqueurs": "American Cordials & Liqueur", "American Distilled Spirits Specialty": "American Distilled Spirit Specialty", "American Vodka": "American Vodkas", "Cocktails / RTD": "Cocktails /RTD", "Imported Cordials & Liqueur": "Imported Cordials & Liqueurs", "Imported Distilled Spirits Specialty": "Imported Distilled Spirit Specialty", "Imported Vodkas": "Imported Vodka", "Temporary &  Specialty Packages": "Temporary & Specialty Packages"}, inplace = True)

#question 1
#group by category name to get each category's total sales
question1 = df.groupby("Category Name").sum()
#sorts records by sales from top to bottom
question1.sort_values(by = "Sale (Dollars)", inplace = True, ascending = False)
#gathering the top five values and applying currency formatting
a = question1["Sale (Dollars)"].apply(lambda x: "${:,.2f}".format(x)).head(5)
#conversion to dataframe for final display
a = pd.DataFrame({"Category Name": a.index, "Sale (Dollars)": a.values}, index = [1,2,3,4,5])
display(a)

#question 2
#creating a new column for quarter
df['Quarter'] = pd.DatetimeIndex(df['Date']).quarter
#filtering out everything except for vodka and whiskey
df_vodka_filter = df[df["Category Name"].str.contains("Vodka", na = False)]
df_whiskey_filter = df[df["Category Name"].str.contains("Whisk", na = False)]
#grouping by quarter
question2vodka = df_vodka_filter.groupby("Quarter").sum()
question2whiskey = df_whiskey_filter.groupby("Quarter").sum()
#getting rid of unnecessary columns
question2vodka = question2vodka['Bottles Sold']
question2whiskey = question2whiskey['Bottles Sold']
#formatting
vb = question2vodka.apply(lambda x: "{:,}".format(x))
wb = question2whiskey.apply(lambda x: "{:,}".format(x))
#conversion to dataframes for display
vb = pd.DataFrame({"Bottles of Vodka Sold": vb.values}, index = ["Q1","Q2","Q3","Q4"])
wb = pd.DataFrame({"Bottles of Whiskey Sold": wb.values}, index = ["Q1","Q2","Q3","Q4"])
display(vb)
display(wb)

#question 3
#creating a new column for month
df['Month'] = pd.DatetimeIndex(df['Date']).month
#grouping by multiple columns, as_index is false so that a multiindex is not created
question3 = df.groupby(['Month','Category Name'], as_index = False)["Bottles Sold"].sum()
#pivot table formation
question3pvt = pd.pivot_table(question3, index = ['Category Name'], columns = ['Month'])
#renaming columns for display
question3pvt.rename(columns = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}, inplace = True)
display(question3pvt)
#takes category names and condenses them into a sorted set
categorieslist = list(question3['Category Name'])
#sorts the list
categorieslist = sorted(categorieslist)
categoriesset = []
#creates the set organically
for i in range(len(categorieslist)):
    if categorieslist[i] not in categoriesset:
        categoriesset.append(categorieslist[i])
positive = []
negative = []
flat = []
#loops through the set
for i in range(len(categoriesset)):
    #creates a filter that only looks at the current category name in the loop
    condition = (question3['Category Name'] == categoriesset[i])
    #created a df that only contains the data pertaining to the specific category
    filteredquestion3vis = question3.loc[condition, ['Category Name','Month','Bottles Sold']]
    #setting up the model for slope calculation
    x = filteredquestion3vis['Month']
    y = filteredquestion3vis['Bottles Sold']
    #computes the slope
    model = np.polyfit(x,y,1)
    #conditionals to check if slope is positive or negative or flat
    if model[0] > 100:
        positive.append(categoriesset[i])
    elif model[0] < -100:
        negative.append(categoriesset[i])
    else:
        flat.append(categoriesset[i])
#printing for the positive, negative, and flat categories
print("Categories that are trending positively (slope > 100):")
for i in range(len(positive)):
    print(positive[i])
print("\nCategories that are trending negatively (slope < -100):")
for i in range(len(negative)):
    print(negative[i])
print("\nCategories that are trending neutrally (-100 < slope < 100):")
for i in range(len(flat)):
    print(flat[i])

#question 4
#filtering for vodka and whiskey categories
question4filter = df['Category Name'].str.contains("Vodka|Whiskey|Whisky|Whiskies", na=False)
# grouping by with filter
question4df = df.loc[question4filter].groupby(["Quarter", "Category Name"], as_index = False)['Sale (Dollars)'].sum()
#pivot table creation
question4pvt = pd.pivot_table(question4df,values = 'Sale (Dollars)', columns = 'Quarter', index = 'Category Name')
#currency formatting
question4pvt.style.format("${:,.2f}")  

,Category Name,Sale (Dollars)
1,American Vodkas,"$51,742,838.89"
2,Canadian Whiskies,"$42,556,225.02"
3,Straight Bourbon Whiskies,"$23,988,780.34"
4,Spiced Rum,"$21,474,888.77"
5,Whiskey Liqueur,"$19,786,836.12"


DataError: No numeric types to aggregate